In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import libem
import numpy as np
from libvis import Visualizations, VideoMaker
import matplotlib.pyplot as plt

from components import *

from scipy.optimize import root_scalar
import realmodel as rm

In [2]:
conv = rm.Conversion(
    mass=1,
    length=0.089916, #cm / sim unit
    time=1,
    charge=1,
    voltage=1
)

SIM_SPACE_SIZE = [int(conv.real_to_sim(dim, "length")) for dim in (7, 7, 7)]
print(SIM_SPACE_SIZE)
SIM_TOPLEFT = [int(conv.real_to_sim(dim, "length")) for dim in (-3.5, -3.5, -3.5)]

RING_RADIUS = 3.25 #cm

particle_conv = rm.Conversion(
    mass=1,
    length=1 / 4.2,
    time=1,
    charge=1,
    voltage=1
)

def generate_barrier_space(V_out=1, V_in=-1, ring_spacing=0.22, ring_fill=2, scale=10):
    sim = libem.EMSimulationSpace3D(space_size=SIM_SPACE_SIZE, top_left=SIM_TOPLEFT, scale=scale)
    cyl1_mask = EMObjectMasks.open_cylinder_in_plane(sim, (-conv.real_to_sim(ring_spacing + (0.45 / 2)), 0, 0),
                                                           conv.real_to_sim(RING_RADIUS - (ring_fill / 2)),
                                                           conv.real_to_sim(ring_fill / 2),
                                                           conv.real_to_sim(0.45),
                                                           0, -1)
    cyl2_mask = EMObjectMasks.open_cylinder_in_plane(sim, (-conv.real_to_sim(ring_spacing / 2), 0, 0),
                                                           conv.real_to_sim(RING_RADIUS - (ring_fill / 2)),
                                                           conv.real_to_sim(ring_fill / 2),
                                                           conv.real_to_sim(0.45),
                                                           0, 1)
    cyl3_mask = EMObjectMasks.open_cylinder_in_plane(sim, (conv.real_to_sim(ring_spacing / 2), 0, 0),
                                                           conv.real_to_sim(RING_RADIUS - (ring_fill / 2)),
                                                           conv.real_to_sim(ring_fill / 2),
                                                           conv.real_to_sim(0.45),
                                                           0, 1)
    
    sim.compute(make_enforcer(
        enf(EMObjects.arbitrary_mask, cyl1_mask, V_out),
        enf(EMObjects.arbitrary_mask, cyl2_mask, V_in),
        enf(EMObjects.arbitrary_mask, cyl3_mask, V_out)
    ))
    sim.get_efield()
    return sim

TIME = 2
SCALE = 1
NPARTICLES = 30
SPACING = 0.22
FILL = RING_RADIUS * 0.75

def sample_particles(n, charge, bounce_coef=None):
    return rm.SimIonBeam("BeamSim.csv", conv.superimpose(particle_conv)).sample(
            num_particles=n, real_charge=charge, bounce=bounce_coef)


[77, 77, 77]


**Simulate Particle Beam**

In [3]:
particles = sample_particles(NPARTICLES, 1, 1)

max_qke = max([abs(p.charge) for p in particles]) * 0.5 * max([p.mass for p in particles]) * \
            (max([np.linalg.norm(p.initial_velocity) for p in particles]))**2

voltage_range = np.arange(0, 2 * max_qke, max_qke / 30)[::-1]
passthrough = np.zeros(voltage_range.shape)
coherence = np.zeros(voltage_range.shape)

fig, axes = plt.subplots(3, 1, figsize=(12, 12))
plt.show()
video = VideoMaker(fig, axes)

print("Computing")

for i, voltage in enumerate(voltage_range):
    sim = generate_barrier_space(0, voltage, SPACING, FILL, SCALE)
    
    passed = []
    passed_v = []
    for j, p in enumerate(particles):
        p.sim = sim
        p.compute_motion((0, (p.initial_location[0] + 1.5) / abs(p.initial_velocity[0])))
        if p.position[0][-1] < -DISTANCE:
            passed.append(p.position[:,-1])
            passed_v.append(p.velocity[:,-1])
        print("\rVoltage " + str(round(voltage, 2)) + ", Particle " + str(j+1).rjust(3), end="")
            
    if len(passed) > 0:
        passed = np.array(passed)
        passed_v = np.array(passed_v)
        passthrough[i] = passed.shape[0] / NPARTICLES
        extrapolate_times = (passed[:,0] + 2.0) / np.abs(passed_v[:,0])
        extrapolated_locs = np.array(
            [(extrapolate_times[i] * passed_v[i,1:]) +  passed[i,1:] for i in range(passed.shape[0])])
        coherence[i] = np.average(np.linalg.norm(extrapolated_locs, axis=1))
    
    video.new_frame()
        
    axes[0].set_xlabel("X")
    axes[0].set_ylabel("Z")
    if voltage != 0:
        sim2d = libem.EMSimulationSpace2D.from_3d(sim, axis=1)
        Visualizations.colormesh_2d(sim2d, color_norm=voltage, graph_ax=axes[0])
    for p in particles:
        Visualizations.trajectory_2d(p.time, p.position, axis=1, graph_ax=axes[0])
    axes[0].set_xlim((-2, 2))
    axes[0].set_ylim((-1, 1))
        
    axes[1].set_xlabel("Voltage")
    axes[1].set_ylabel("Passthrough Fraction")
    axes[1].set_xlim((voltage_range[0], voltage_range[-1]))
    axes[1].set_ylim((0, 1))
    axes[1].plot(np.array(voltage_range[:i+1]), np.array(passthrough[:i+1]), color="m")
    
    axes[2].set_xlabel("Voltage")
    axes[2].set_ylabel("Coherence Distance")
    axes[2].set_xlim((voltage_range[0], voltage_range[-1]))
    axes[2].set_ylim((0, max(max(coherence[:i+1]), 1) * 1.1))
    axes[2].plot(np.array(voltage_range[:i+1]), np.array(coherence[:i+1]), color="g")
        
    video.draw_frame()
    
video.make_movie("beam_passthrough_rings.mp4")

<IPython.core.display.Javascript object>

Computing


KeyboardInterrupt: 